based on https://pmc.ncbi.nlm.nih.gov/articles/PMC8871265/

In [ ]:
import numpy as np
import time
import os
from tensorflow.keras.applications import MobileNetV2, NASNetMobile
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score
import joblib

In [ ]:
# Paths to save model checkpoints
mobilenet_weights_path = 'models/mobilenet_weights.h5'
nasnet_weights_path = 'models/nasnet_weights.h5'

# Load MobileNetV2 and NASNetMobile without the top layers
mobilenet_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
nasnet_base = NASNetMobile(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add Global Average Pooling to convert the convolutional feature maps to a 1D vector
mobilenet_model = Model(inputs=mobilenet_base.input, outputs=GlobalAveragePooling2D()(mobilenet_base.output))
nasnet_model = Model(inputs=nasnet_base.input, outputs=GlobalAveragePooling2D()(nasnet_base.output))

# Load weights if available
if os.path.exists(mobilenet_weights_path):
    print("Loading MobileNetV2 weights from checkpoint...")
    mobilenet_model.load_weights(mobilenet_weights_path)
else:
    print("No MobileNetV2 weights found. Proceeding without loading.")

if os.path.exists(nasnet_weights_path):
    print("Loading NASNetMobile weights from checkpoint...")
    nasnet_model.load_weights(nasnet_weights_path)
else:
    print("No NASNetMobile weights found. Proceeding without loading.")

In [ ]:
# Create an ImageDataGenerator instance with data augmentation
datagen = ImageDataGenerator(
    rotation_range=90,          # Randomly rotate images in multiples of 90 degrees
    zoom_range=0.5,            # Randomly zoom images by up to 50%
    horizontal_flip=True,       # Randomly flip images horizontally
    vertical_flip=True          # Randomly flip images vertically
)

# Generate batches from the NumPy array along with labels
generator = datagen.flow(
    images,
    labels,
    batch_size=8
)

# Example to iterate through generated batches
for batch_images, batch_labels in generator:
    # Use the batch for training, prediction, etc.
    print("Generated batch shape:", batch_images.shape)
    print("Generated labels shape:", batch_labels.shape)
    break

In [ ]:
# Extract features using both models
mobilenet_features = mobilenet_model.predict(train_generator)
nasnet_features = nasnet_model.predict(train_generator)

In [ ]:
# Save model weights after feature extraction for future resumption
mobilenet_model.save_weights(mobilenet_weights_path)
nasnet_model.save_weights(nasnet_weights_path)

In [ ]:
# Fuse features by concatenating them
fused_features = np.concatenate([mobilenet_features, nasnet_features], axis=1)
labels = train_generator.classes

# Split the data into 80% training and 20% testing set
X_train, X_test, y_train, y_test = train_test_split(fused_features, labels, test_size=0.2, random_state=42)

# Standardize the features and train an SVM model
scaler = StandardScaler()
svm_model = SVC(kernel='poly', degree=3, C=1.0, probability=True)

# Combine the scaler and SVM into a pipeline
svm_pipeline = make_pipeline(scaler, svm_model)

# Check if a previously trained model exists to resume training
model_path = 'cubic_svm_model.pkl'
if os.path.exists(model_path):
    print("Loading existing SVM model to resume training...")
    svm_pipeline = joblib.load(model_path)
else:
    print("No existing SVM model found. Training a new model...")

In [ ]:
# Train the SVM model using the training set
start_time = time.time()
svm_pipeline.fit(X_train, y_train)
training_time = time.time() - start_time

In [ ]:
# Save the trained SVM model for future use
joblib.dump(svm_pipeline, model_path)

In [ ]:
# Example evaluation on the test set
predictions = svm_pipeline.predict(X_test)
prediction_probabilities = svm_pipeline.predict_proba(X_test)[:, 1]

In [ ]:
# Metrics
accuracy = accuracy_score(y_test, predictions) * 100
precision = precision_score(y_test, predictions) * 100
recall = recall_score(y_test, predictions) * 100
f1 = f1_score(y_test, predictions) * 100
auc = roc_auc_score(y_test, prediction_probabilities)
conf_matrix = confusion_matrix(y_test, predictions)
kappa = cohen_kappa_score(y_test, predictions)
specificity = conf_matrix[0][0] / (conf_matrix[0][0] + conf_matrix[0][1]) * 100 if (conf_matrix[0][0] + conf_matrix[0][1]) != 0 else 0

print(f'Test Accuracy (%): {accuracy:.2f}')
print(f'Precision (%): {precision:.2f}')
print(f'Sensitivity (Recall) (%): {recall:.2f}')
print(f'F1-Score (%): {f1:.2f}')
print(f'AUC: {auc:.2f}')
print(f'Specificity (%): {specificity:.2f}')
print(f'Cohen Kappa: {kappa:.2f}')
print(f'Training Time (s): {training_time:.2f}')